### Compositing

#### Camadas de frente e de fundo
- Utiliza-se a cobertura do pixel (alpha): Fração do pixel coberta pela camada da frente
- OBS: alpha será a quarta componente de um pixel(RGBA) variando de 0(transparente) a 255(totalmente opaco)

In [1]:
import random

alpha = float("{:.2f}".format(random.random()))

cf = cb = 1.0

c = alpha*cf + (1-alpha)*cb

print(alpha, cf, cb, c)

0.27 1.0 1.0 1.0


### Negativo de Imagens
 - Função de transformação dada por $s = T(r) = L -1 -r$

In [ ]:
# para um exemplo com pixel de tom 64, onde L é o tom máximo da escala de cinza (256)
r = 64
L = 256
neg = (L - 1) - r
neg

In [1]:
%matplotlib inline

import numpy as np


def load_image(image_url):
    header = []
    with open(image_url, 'r') as f:
        for l in range(4):
            header.append(f.readline())
        x, y = [int(n) for n in header[2].split()]
        pixels = np.zeros((x, y))
        for j in range(x):
            for k in range(y):
                pixels[j][k] = int(f.readline())
        return header, pixels.astype('int64')

3. Para cada operação pontual abaixo, crie um método que aplique esta operação em uma imagem e salve o resultado em uma nova imagem:
    - Negativo (recebe o nome do arquivo .ppm como entrada);
    - Limiarização (recebe o nome do arquivo .ppm e o limiar como entrada).

4. Testar o que fizeram e verificar o efeito dos operadores nas imagens de entrada. Seguem algumas em anexo a esta atividade, mas vocês podem exportar outras para testar seus operadores (exportar como ppm em ASCII).

In [7]:
url = 'matriz.ppm'

# header, pixels = load_image('Figuras/barco.pgm')
header, pixels = load_image('matriz.ppm')
# print(header, pixels)
middle_value = int(header[3]) // 2
def store_image(image_url, transform_func=None, threshold=middle_value):
    with open(image_url, 'w') as f:
        [f.write(l) for l in header]
        x, y = [int(n) for n in header[2].split()]
        LMax = int(header[3]) + 1
        if transform_func == 'equalize':
            equalize([pixels, LMax])
        for j in range(x):
            for k in range(y):
                # 3th - Negative filter
                if transform_func == 'negative':
                    pixels[j][k] = (LMax - 1) - pixels[j][k]
                # 3th - Thresholding filter
                if transform_func == 'threshold':
#                     pixels[j][k] = (lambda: 0, lambda: int(header[3]))[pixels[j][k] >= threshold]()
                    pixels[j][k] = int(header[3]) if pixels[j][k] >= threshold else 0                    
                f.write(str(int(pixels[j][k]))+'\n')
        return 0

if store_image(url, 'equalize') == 0: print('Operation Successful')

Operation Successful


In [ ]:
# def level_transform(pixels, LMax, transformations=[]):
#     if len(transformations) == 0: return
#     for transform_func in transformations:
#         if transform_func == 'negative': 
#                 pixels[j][k] = ((int(header[3])) - pixels[j][k]) # header[3] is already L-1
#         if transform_func == 'threshold':
#                 pixels[j][k] = int(header[3]) if pixels[j][k] >= threshold else 0

In [ ]:
# Bring down the transform functions to a proper function like that:
"""
    transform=None # expects array of object strings
    provide it as below:
    
    def level_transform(transformations=[]):
        for transform_func in transformations:
            if transform_func == 'negative': 
                    pixels[j][k] = ((int(header[3])) - pixels[j][k]) # header[3] is already L-1
            if transform_func == 'threshold':
                    pixels[j][k] = int(header[3]) if pixels[j][k] >= threshold else 0
"""

## Histograma é dado por
$ h(r_k) = n_k $, onde $r_k$ é a tonalidade da imagem com k = 0,1,2...L-1 e $n_k$ é o número de pixels na tonalidade k

In [2]:
# Histogram of the picture
def h(img_data):
    LMax = int(header[3]) + 1
    H = np.zeros(LMax).astype('int64')
    for x in range(img_data.shape[0]):
        for y in range(img_data.shape[1]):
            H[img_data[x][y]] += 1
    return H

### Normalização de histograma é dada por
$p(r_k) = n_k/M.N$, onde M e N representam o número total de pixels da imagem

In [3]:
def normalize(histogram, resolution):
    M, N = resolution
    return histogram / (M*N)

In [4]:
### Cummulative summation of the probabilities

# P.sum() # Summation of the K_(L-1) probabilities
def cumsum(P):
    C = np.zeros_like(P)
    for i in range(C.size):
        cumsum = 0
        for k in range(i+1):
            cumsum += P[k]
        C[i] = cumsum
    return C

In [5]:
def equalize(img_data=[]):
    if len(img_data) != 2: return
    
    pixels, LMax = img_data
    resolution = pixels.shape
    histogram = h(pixels)
    # Normalize
    P = normalize(histogram, resolution)
    # Sum Probabilities
    C = cumsum(P)
    # Map new indexes
    eq = LMax - 1
    equalized_histogram = np.array(
        np.round(eq * C)
    ).astype('int64')
    # equalize each pixel
    for i in range(pixels.shape[0]):
        for j in range(pixels.shape[1]):
            pixels[i][j] = equalized_histogram[pixels[i][j]]

### Drafts

In [ ]:
header, pixels = load_image('Figuras/lua.pgm')

resolution = [int(n) for n in header[2].split()]
# pixels.shape
h(pixels)


In [ ]:
# histogram = h(pixels)
# P = normalize(histogram, resolution)
# P, P.size

In [ ]:
# Draft - Executing cumsum examples

# LMax = int(header[3]) + 1

# C = np.zeros_like(P)
# for i in range(C.size):
#     C[i] = cumsum(i)

# cumsum(LMax - 1)
# C

In [ ]:
### Equalizing histograms

eq = LMax - 1
eq_histogram = np.array(eq * C).astype('int64')
eq_histogram